In [1]:
import numpy as np

In [8]:
np.outer(np.arange(1,10).reshape((3,3)),np.arange(1,10).reshape((3,3)))

array([[ 1,  2,  3,  4,  5,  6,  7,  8,  9],
       [ 2,  4,  6,  8, 10, 12, 14, 16, 18],
       [ 3,  6,  9, 12, 15, 18, 21, 24, 27],
       [ 4,  8, 12, 16, 20, 24, 28, 32, 36],
       [ 5, 10, 15, 20, 25, 30, 35, 40, 45],
       [ 6, 12, 18, 24, 30, 36, 42, 48, 54],
       [ 7, 14, 21, 28, 35, 42, 49, 56, 63],
       [ 8, 16, 24, 32, 40, 48, 56, 64, 72],
       [ 9, 18, 27, 36, 45, 54, 63, 72, 81]])

In [13]:
p = np.outer([1,-1,1,-1,-1],[1,-1,1,-1,-1])
print(p)
np.fill_diagonal(p,0)
p

[[ 1 -1  1 -1 -1]
 [-1  1 -1  1  1]
 [ 1 -1  1 -1 -1]
 [-1  1 -1  1  1]
 [-1  1 -1  1  1]]


array([[ 0, -1,  1, -1, -1],
       [-1,  0, -1,  1,  1],
       [ 1, -1,  0, -1, -1],
       [-1,  1, -1,  0,  1],
       [-1,  1, -1,  1,  0]])

In [9]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:

class HopfieldNet(object):

    def __init__(self, num_units):
        self.num_units = num_units
        self.w = np.zeros((num_units, num_units))
        self.b = np.zeros(num_units)

    def store(self, data):
        data = _data.reshape(data.shape[0], 1)
        activations = np.dot(data, data.T)
        np.fill_diagonal(activations, 0)  # because there are no connections to itself
        self.w += activations
        self.b += data.ravel()

    def get_energy(self, data):
        # first let's again compute product of activations
        data = _data.reshape(data.shape[0], 1)
        activations = np.float32(np.dot(data, data.T))
        np.fill_diagonal(activations, 0)
        # then multiply each activation by a weight elementwise
        activations *= self.w
        # total energy consists of weight and bias term
        weight_term = np.sum(activations) / 2  # divide by 2, because we've counted neurons twice
        bias_term = np.dot(self.b, data)[0]
        return - bias_term - weight_term
    
    def restore(self, data):
        data = np.copy(_data)
        idx = range(len(data))
        # make 10 passes through the data
        for i in xrange(10):
            for _ in xrange(len(data)):
                j = np.random.choice(idx)
                inputs = np.sum(data * self.w[j])
                if inputs > 0:
                    data[j] = 1
                else:
                    data[j] = -1
        return data

In [6]:
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np

In [1]:
from keras.layers import Dense, Input, Lambda
from keras.models import Model,Sequential
from keras.optimizers import Adam
from keras.layers.merge import Concatenate
from keras import backend as K
from keras import metrics

Using TensorFlow backend.


In [27]:
def vae_feature_generation(data,components='auto',funnel=(2.5,9),
                           batch=200,epoch = 250,epsilon=1.0,opt= None,
                           validation=False, plot_it=False):
    
    original_dim = data.shape[1]

    if components == 'auto':
        intermediate_dim = int(np.ceil(original_dim/2.5))
        latent_dim = int(np.ceil(intermediate_dim/9))
    elif components == 'ratio':
        intermediate_dim = int(np.ceil(original_dim/funnel[0]))
        latent_dim = int(np.ceil(intermediate_dim/funnel[1]))
    elif components == 'values':
        intermediate_dim = funnel[0]
        latent_dim = funnel[1]
    
    batch_size = batch
    epochs = epoch
    epsilon_std = epsilon

    x = Input(shape = (original_dim,))
    h = Dense(intermediate_dim, activation='elu')(x)
    z_mean = Dense(latent_dim)(h)
    z_log_var = Dense(latent_dim)(h)


    def sampling(args):
        z_mean, z_log_var = args
        epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim), mean=0.,
                                  stddev=epsilon_std)
        return z_mean + K.exp(z_log_var / 2) * epsilon

    # note that "output_shape" isn't necessary with the TensorFlow backend
    z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

    # we instantiate these layers separately so as to reuse them later
    decoder_h = Dense(intermediate_dim, activation='elu')
    decoder_mean = Dense(original_dim, activation='relu')
    h_decoded = decoder_h(z)
    x_decoded_mean = decoder_mean(h_decoded)
    
    # instantiate VAE model
    vae = Model(inputs=x, outputs=x_decoded_mean)

    # Compute VAE loss
    xent_loss = original_dim * metrics.mean_squared_error(x, x_decoded_mean)
    kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    vae_loss = K.mean(xent_loss + kl_loss)

    vae.add_loss(vae_loss)
    if opt is None:
        opt = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
    vae.compile(optimizer=opt)
    vae.summary()
    
    # fitting our generated features on the data
    vae.fit(data,
            shuffle=True,
            epochs=epochs,
            batch_size=batch_size
    #         validation_data=(x_test, None)
           )
    
    # Encoder model for feature generation
    encoder = Model(inputs=x, outputs = z_mean)

    # display a 2D plot of the digit classes in the latent space
    tr = encoder.predict(data, batch_size=batch_size)
    if plot_it:
        print('\nPlotting features!\n')
        # Plotting with SNS
        sns.set(style="ticks")
        df_plot = pd.DataFrame(tr)
        g = sns.PairGrid(df_plot)
        g = g.map_upper(plt.scatter)
        g = g.map_lower(sns.kdeplot, cmap="Blues_d")
        g = g.map_diag(sns.kdeplot, lw=3, legend=False)
        g
        
    return tr, latent_dim, encoder


In [26]:
data = load_iris()
df  = pd.concat([pd.DataFrame(data.data),pd.DataFrame(data.target)],axis = 1)
data.feature_names.append('target')
df.columns = data.feature_names
print(df.head())
df.describe()

   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                5.1               3.5                1.4               0.2   
1                4.9               3.0                1.4               0.2   
2                4.7               3.2                1.3               0.2   
3                4.6               3.1                1.5               0.2   
4                5.0               3.6                1.4               0.2   

   target  
0       0  
1       0  
2       0  
3       0  
4       0  


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667,1.000000
std,0.828066,0.433594,1.764420,0.763161,0.819232
min,4.300000,2.000000,1.000000,0.100000,0.000000
25%,5.100000,2.800000,1.600000,0.300000,0.000000
50%,5.800000,3.000000,4.350000,1.300000,1.000000
75%,6.400000,3.300000,5.100000,1.800000,2.000000
max,7.900000,4.400000,6.900000,2.500000,2.000000


In [14]:
df.iloc[:4,3]

0    0.2
1    0.2
2    0.2
3    0.2
Name: petal width (cm), dtype: float64

In [28]:
vae_feature_generation(df.iloc[:,0:4],components='values',funnel=(20,2),epoch=20)

/Users/jitins_lab/anaconda2/envs/hyper/lib/python3.6/site-packages/ipykernel/__main__.py:53: UserWarning: Output "dense_30" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "dense_30" during training.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 4)            0                                            
__________________________________________________________________________________________________
dense_26 (Dense)                (None, 20)           100         input_6[0][0]                    
__________________________________________________________________________________________________
dense_27 (Dense)                (None, 2)            42          dense_26[0][0]                   
__________________________________________________________________________________________________
dense_28 (Dense)                (None, 2)            42          dense_26[0][0]                   
__________________________________________________________________________________________________
lambda_6 (

(array([[  2.03996205e+00,  -4.29239750e-01],
        [  1.96641374e+00,  -4.00960177e-01],
        [  1.85129476e+00,  -4.32687491e-01],
        [  1.82796276e+00,  -3.71154815e-01],
        [  1.98720014e+00,  -4.30590540e-01],
        [  2.21718717e+00,  -4.18170601e-01],
        [  1.81141949e+00,  -4.32150632e-01],
        [  2.00577116e+00,  -3.95570070e-01],
        [  1.73306429e+00,  -3.83179754e-01],
        [  1.96086860e+00,  -3.57192546e-01],
        [  2.18510985e+00,  -4.18821007e-01],
        [  1.91845453e+00,  -3.63307297e-01],
        [  1.90867829e+00,  -3.76680672e-01],
        [  1.63708973e+00,  -4.46803421e-01],
        [  2.33702159e+00,  -5.20943105e-01],
        [  2.32602048e+00,  -4.96995658e-01],
        [  2.17945790e+00,  -5.24488389e-01],
        [  2.05036569e+00,  -4.51263934e-01],
        [  2.35281157e+00,  -4.00747567e-01],
        [  2.04662704e+00,  -4.36110109e-01],
        [  2.21344256e+00,  -3.53012174e-01],
        [  2.06154585e+00,  -4.542

In [32]:
df = pd.read_csv('/Users/jitins_lab/Documents/work/demo_data.csv')

In [33]:
df.head()

,no_days_membership,no_days_last_seen,email_levels_aol.com,email_levels_comcast.net,email_levels_gmail.com,email_levels_hotmail.com,email_levels_mail.house.gov,email_levels_others,email_levels_yahoo.com,browser_Chrome,...,region_DC,region_FL,region_IL,region_MD,region_NY,region_PA,region_TX,region_VA,region_others,clusters
0,53,36,0,0,1,0,0,0,0,1,...,0,0,1,0,0,0,0,0,0,2
1,514,39,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,514,39,0,0,0,0,0,1,0,1,...,0,0,0,0,1,0,0,0,0,0
3,199,36,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,7
4,514,47,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [36]:
vae_feature_generation(df.iloc[:,2:],components='values',funnel=(20,5),epoch=20)

/Users/jitins_lab/anaconda2/envs/hyper/lib/python3.6/site-packages/ipykernel/__main__.py:53: UserWarning: Output "dense_60" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "dense_60" during training.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           (None, 40)           0                                            
__________________________________________________________________________________________________
dense_56 (Dense)                (None, 20)           820         input_12[0][0]                   
__________________________________________________________________________________________________
dense_57 (Dense)                (None, 5)            105         dense_56[0][0]                   
__________________________________________________________________________________________________
dense_58 (Dense)                (None, 5)            105         dense_56[0][0]                   
__________________________________________________________________________________________________
lambda_12 

(array([[ 0.1406091 , -0.06560794, -0.44563228,  0.40716445,  0.18203133],
        [-0.96386069, -0.04932985, -0.0777006 , -1.4933871 , -0.06372079],
        [ 0.96305639,  0.04226241,  0.09615929, -0.42803565,  0.04171118],
        ..., 
        [-0.81237823, -0.03154868,  0.06155227,  0.42499986, -0.01237265],
        [-1.10280693,  0.02248526,  0.05725055,  0.41712466, -0.05142437],
        [-0.98738986, -0.0550078 , -0.03147643, -1.26622903, -0.07907742]], dtype=float32),
 5,
 <keras.engine.training.Model at 0x1a21a23d68>)

In [24]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
no_days_membership,10100.0,393.572970,161.142717,35.0,259.0,514.0,514.0,543.0
no_days_last_seen,10100.0,41.587327,8.196698,34.0,39.0,40.0,45.0,391.0
email_levels_aol.com,10100.0,0.039901,0.195736,0.0,0.0,0.0,0.0,1.0
email_levels_comcast.net,10100.0,0.024752,0.155378,0.0,0.0,0.0,0.0,1.0
email_levels_gmail.com,10100.0,0.251287,0.433775,0.0,0.0,0.0,1.0,1.0
email_levels_hotmail.com,10100.0,0.031089,0.173567,0.0,0.0,0.0,0.0,1.0
email_levels_mail.house.gov,10100.0,0.139802,0.346799,0.0,0.0,0.0,0.0,1.0
email_levels_others,10100.0,0.450495,0.497568,0.0,0.0,0.0,1.0,1.0
email_levels_yahoo.com,10100.0,0.062673,0.242386,0.0,0.0,0.0,0.0,1.0
browser_Chrome,10100.0,0.251782,0.434058,0.0,0.0,0.0,1.0,1.0


In [86]:
def vae_feature_generation(data,components='auto',funnel=(45,30,15,10),
                           batch=200,epoch = 250,epsilon=1.0,opt= None,
                           validation=False, plot_it=False):
    
    original_dim = data.shape[1]

    if components == 'auto':
        intermediate_dim_1 = 45
        intermediate_dim_2 = 30
#         intermediate_dim_3 = 15
        latent_dim = 10
    elif components == 'ratio':
        intermediate_dim_1 = int(np.ceil(original_dim/funnel[0]))
        intermediate_dim_2 = int(np.ceil(original_dim/funnel[1]))
#         intermediate_dim_3 = int(np.ceil(original_dim/funnel[2]))
        latent_dim = int(np.ceil(intermediate_dim/funnel[3]))
    elif components == 'values':
        intermediate_dim_1 = funnel[0]
        intermediate_dim_2 = funnel[1]
#         intermediate_dim_3 = funnel[2]
        latent_dim = funnel[3]
    
    batch_size = batch
    epochs = epoch
    epsilon_std = epsilon

    x = Input(shape = (original_dim,))
    h_1 = Dense(intermediate_dim_1, activation='elu')(x)
    h_2 = Dense(intermediate_dim_2, activation='elu')(h_1)
#     h_3 = Dense(intermediate_dim_3, activation='elu')(h_1)
    z_mean = Dense(latent_dim)(h_2)
    z_log_var = Dense(latent_dim)(h_2)


    def sampling(args):
        z_mean, z_log_var = args
        epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim), mean=0.,
                                  stddev=epsilon_std)
        return z_mean + K.exp(z_log_var / 2) * epsilon

    # note that "output_shape" isn't necessary with the TensorFlow backend
    z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

    # we instantiate these layers separately so as to reuse them later
    decoder_h_2 = Dense(intermediate_dim_2, activation='elu')
    decoder_h_1 = Dense(intermediate_dim_1, activation='elu')
    decoder_mean = Dense(original_dim, activation='relu')
    h_decoded_2 = decoder_h_2(z)
    h_decoded_1 = decoder_h_1(h_decoded_2)
    x_decoded_mean = decoder_mean(h_decoded_1)
    
    # instantiate VAE model
    vae = Model(inputs=x, outputs=x_decoded_mean)

    # Compute VAE loss
    xent_loss = original_dim * metrics.mean_squared_error(x, x_decoded_mean)
    kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    vae_loss = K.mean(xent_loss + kl_loss)

    vae.add_loss(vae_loss)
    if opt is None:
        opt = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
    vae.compile(optimizer=opt)
    vae.summary()
    
    # fitting our generated features on the data
    vae.fit(data,
            shuffle=True,
            epochs=epochs,
            batch_size=batch_size
    #         validation_data=(x_test, None)
           )
    
    # Encoder model for feature generation
    encoder = Model(inputs=x, outputs = z_mean)

    # display a 2D plot of the digit classes in the latent space
    tr = encoder.predict(data, batch_size=batch_size)
    if plot_it:
        print('\nPlotting features!\n')
        # Plotting with SNS
        sns.set(style="ticks")
        df_plot = pd.DataFrame(tr)
        g = sns.PairGrid(df_plot)
        g = g.map_upper(plt.scatter)
        g = g.map_lower(sns.kdeplot, cmap="Blues_d")
        g = g.map_diag(sns.kdeplot, lw=3, legend=False)
        g
        
    return tr, latent_dim, encoder


In [89]:
vae_feature_generation(df.iloc[:,:4],components='values',funnel=(9,5,3,3),epoch=20)

/Users/jitins_lab/anaconda2/envs/hyper/lib/python3.6/site-packages/ipykernel/__main__.py:63: UserWarning: Output "dense_181" missing from loss dictionary. We assume this was done on purpose, and we will not be expecting any data to be passed to "dense_181" during training.


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 4)            0                                            
__________________________________________________________________________________________________
dense_175 (Dense)               (None, 9)            45          input_2[0][0]                    
__________________________________________________________________________________________________
dense_176 (Dense)               (None, 5)            50          dense_175[0][0]                  
__________________________________________________________________________________________________
dense_177 (Dense)               (None, 3)            18          dense_176[0][0]                  
__________________________________________________________________________________________________
dense_178 

(array([[-1.1596504 , -0.92211324, -0.6296273 ],
        [-1.1672746 , -0.94965315, -0.59715897],
        [-1.1029255 , -0.8750694 , -0.5854537 ],
        [-1.1309884 , -0.897882  , -0.57688224],
        [-1.1358716 , -0.8907459 , -0.6238425 ],
        [-1.2461673 , -0.9778152 , -0.65713376],
        [-1.1025052 , -0.8484813 , -0.5793726 ],
        [-1.1693503 , -0.93064326, -0.619831  ],
        [-1.0978465 , -0.87497944, -0.55111897],
        [-1.162996  , -0.94648784, -0.6081832 ],
        [-1.2082036 , -0.96418935, -0.6607308 ],
        [-1.1539891 , -0.9081588 , -0.60489875],
        [-1.135788  , -0.9287118 , -0.5952572 ],
        [-1.0050159 , -0.7954196 , -0.54359055],
        [-1.1956593 , -0.956868  , -0.6961391 ],
        [-1.218105  , -0.9449584 , -0.69072086],
        [-1.1842457 , -0.92383933, -0.65234125],
        [-1.1735643 , -0.92746913, -0.6238568 ],
        [-1.2957125 , -1.0359349 , -0.6822718 ],
        [-1.1633693 , -0.9030027 , -0.63393253],
        [-1.2696844 

### Flexible VAE latent features generator

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [3]:
from sklearn.datasets import load_iris
import numpy as np
import pandas as pd

In [4]:
from keras.layers import Dense, Input, Lambda
from keras.models import Model,Sequential
from keras.optimizers import Adam
from keras.layers.merge import Concatenate
from keras import backend as K
from keras import metrics

/Users/jitins_lab/anaconda2/envs/hyper/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [17]:
funnel[:-1]

(30, 14)

In [85]:
original_dim = 4
funnel=(9,5,3)

mod_e = Sequential(name='encoder')
mod_e.add(Dense(funnel[0], input_dim=original_dim))
for i in funnel[1:]:
    mod_e.add(Dense(i, activation='elu'))
                
mod_e.summary()
z_mean = mod_e.output
z_log_var = mod_e.output

# batch_size = batch
# epochs = epoch
epsilon_std = 0.1

def sampling(x,grps):
    z_mean, z_log_var = grps
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], funnel[-1]), mean=0.,
                              stddev=epsilon_std)
    return z_mean + K.exp(z_log_var / 2) * epsilon

z = Lambda(sampling, output_shape=(funnel[-1],),arguments={'grps':[z_mean, z_log_var]})
    
mod_d = Sequential(name='decoder')
for i,v in enumerate(reversed(funnel[:-1])):
    if i == 0:
        mod_d.add(Dense(v, input_dim=funnel[-1],activation='elu'))
    else:
        mod_d.add(Dense(v, activation='elu'))
# mod_d = Model(inputs=inp,outputs=z)
mod_d.add(Dense(original_dim, activation='elu'))
mod_d.summary()

mod = Sequential()
mod.add(mod_e)
mod.add(z)
mod.add(mod_d)
mod.summary()

def vae_loss(y_true, y_pred):
#         x = K.flatten(x)
#         x_decoded_mean = K.flatten(x_decoded_mean)
    print(y_true,y_pred)    
    xent_loss = original_dim * metrics.mean_squared_error(y_true, y_pred)
    kl_loss = - 0.5 * K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
    return xent_loss + kl_loss
mod.compile(optimizer='adam', loss=vae_loss)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_162 (Dense)            (None, 9)                 45        
_________________________________________________________________
dense_163 (Dense)            (None, 5)                 50        
_________________________________________________________________
dense_164 (Dense)            (None, 3)                 18        
Total params: 113
Trainable params: 113
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_165 (Dense)            (None, 5)                 20        
_________________________________________________________________
dense_166 (Dense)            (None, 9)                 54        
_________________________________________________________________
dense_167 (D

In [84]:
mod.fit(df.iloc[:,:4].values)

AttributeError: 'NoneType' object has no attribute 'shape'

In [73]:
def vae_latent_feature(data,components='auto',funnel=(2.5,9),
                           batch=200,epoch = 250,epsilon=1.0,opt= None,
                           validation=False, plot_it=False):
    
    original_dim = data.shape[1]
    
    batch_size = batch
    epochs = epoch
    epsilon_std = epsilon
    
    # making encoder
    mod_e = Sequential(name='encoder')
    mod_e.add(Dense(funnel[0], input_dim=original_dim))
    for i in funnel[1:]:
        mod_e.add(Dense(i, activation='elu'))

    mod_e.summary()
    z_mean = mod_e.output
    z_log_var = mod_e.output

    def sampling(x,grps):
        z_mean, z_log_var = grps
        epsilon = K.random_normal(shape=(K.shape(z_mean)[0], funnel[-1]), mean=0.,
                                  stddev=epsilon_std)
        return z_mean + K.exp(z_log_var / 2) * epsilon
    
    # making sampling layer
    z = Lambda(sampling, output_shape=(funnel[-1],),arguments={'grps':[z_mean, z_log_var]})
    
    #making decoder
    mod_d = Sequential(name='decoder')
    for i,v in enumerate(reversed(funnel[:-1])):
        if i == 0:
            mod_d.add(Dense(v, input_dim=funnel[-1],activation='elu'))
        else:
            mod_d.add(Dense(v, activation='elu'))
    mod_d.add(Dense(original_dim, activation='elu'))
    mod_d.summary()
    
    
    # instantiate VAE model
    vae = Sequential()
    vae.add(mod_e)
    vae.add(z)
    vae.add(mod_d)

    # Compute VAE loss
#     xent_loss = original_dim * metrics.mean_squared_error(mod_e.get_layer(index=1).input, mod_d.get_output_at(len(mod_d.layers)-1))
#     kl_loss = - 0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
#     vae_loss = K.mean(xent_loss + kl_loss)
    
    def vae_loss(y_true, y_pred):
#         x = K.flatten(x)
#         x_decoded_mean = K.flatten(x_decoded_mean)
        xent_loss = original_dim * metrics.mean_squared_error(y_true, y_pred)
        kl_loss = - 0.5 * K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
        return xent_loss + kl_loss

#     vae.add_loss(vae_loss)
    if opt is None:
        opt = Adam(lr=0.01, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)
    vae.compile(optimizer='adam', loss=vae_loss)
    vae.summary()
    
    # fitting our generated features on the data
    vae.fit(data,
#             shuffle=True,
            epochs=50,
            batch_size=10
    #         validation_data=(x_test, None)
           )
    
    # Encoder model for feature generation
    encoder = mod_e

    # display a 2D plot of the digit classes in the latent space
    tr = encoder.predict(data, batch_size=batch_size)
    if plot_it:
        print('\nPlotting features!\n')
        # Plotting with SNS
        sns.set(style="ticks")
        df_plot = pd.DataFrame(tr)
        g = sns.PairGrid(df_plot)
        g = g.map_upper(plt.scatter)
        g = g.map_lower(sns.kdeplot, cmap="Blues_d")
        g = g.map_diag(sns.kdeplot, lw=3, legend=False)
        g
        
    return tr, latent_dim, encoder


In [74]:
data = load_iris()
df  = pd.concat([pd.DataFrame(data.data),pd.DataFrame(data.target)],axis = 1)
data.feature_names.append('target')
df.columns = data.feature_names
print(df.head())
df.describe()

   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                5.1               3.5                1.4               0.2   
1                4.9               3.0                1.4               0.2   
2                4.7               3.2                1.3               0.2   
3                4.6               3.1                1.5               0.2   
4                5.0               3.6                1.4               0.2   

   target  
0       0  
1       0  
2       0  
3       0  
4       0  


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667,1.000000
std,0.828066,0.433594,1.764420,0.763161,0.819232
min,4.300000,2.000000,1.000000,0.100000,0.000000
25%,5.100000,2.800000,1.600000,0.300000,0.000000
50%,5.800000,3.000000,4.350000,1.300000,1.000000
75%,6.400000,3.300000,5.100000,1.800000,2.000000
max,7.900000,4.400000,6.900000,2.500000,2.000000


In [75]:
vae_latent_feature(data=df.iloc[:,0:4],funnel=(7,2),batch=10)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_134 (Dense)            (None, 7)                 35        
_________________________________________________________________
dense_135 (Dense)            (None, 2)                 16        
Total params: 51
Trainable params: 51
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_136 (Dense)            (None, 7)                 21        
_________________________________________________________________
dense_137 (Dense)            (None, 4)                 32        
Total params: 53
Trainable params: 53
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)      

AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
print(plt.style.available)